In [1]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n64", 5555)

connected to arkouda server tcp://*:5555


In [3]:
src_main = [1, 2, 1, 3, 4, 2, 5, 4, 5, 6, 3, 8, 9, 4, 9, 10]
dst_main = [0, 1, 3, 4, 1, 4, 2, 5, 6, 7, 8, 9, 3, 9, 10, 4]
src_maingraph = ak.array(src_main)
dst_maingraph = ak.array(dst_main)

maingraph_nodes = list(set(src_main) | set(dst_main))
maingraph_nodes.sort()

labels1_maingraph = ak.array(["yellow", "red", "blue", "blue", "blue", "red", "blue","yellow", "blue", "yellow", "blue"])
labels2_maingraph = ak.array([10, 12, 10, 11, 11, 12, 11, 13, 10, 13, 10])
labels3_maingraph = ak.array([False, True, True, True, True, True, False, False, True, False, False])
labels4_maingraph = ak.Categorical(labels1_maingraph)

rels1_maingraph = ak.array([10, 5, 10, 10, 10, 10, 10, 10, 10, 5, 5, 10, 10, 5, 5, 10])

edge_df = ak.DataFrame({"src":src_maingraph, "dst":dst_maingraph,
                        "value":rels1_maingraph 
})
node_df = ak.DataFrame({"nodes":ak.array(maingraph_nodes),
                        "color":labels1_maingraph,
                        "value":labels2_maingraph,
                        "True?":labels3_maingraph,
                        "cats":labels4_maingraph
})

In [4]:
pg = ar.PropGraph()
pg.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
pg.load_node_attributes(node_df, node_column="nodes")

In [5]:
src_list = [0, 1, 2, 1]
dst_list = [1, 2, 0, 3]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

labels1_subgraph = ak.array(["blue", "blue", "red", "yellow"])
labels2_subgraph = ak.array([10, 11, 12, 13])
labels3_subgraph = ak.array([True, True, True, False])
labels4_subgraph = ak.Categorical(labels1_subgraph)

rels1_subgraph = ak.array([10, 10, 10, 5])

In [6]:
"""VF2-SI STRUCTURAL"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = None, algorithm_type = "si", 
                                           reorder = True, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with reordering")

We found 17.0 monos inside of the graph with reordering


In [7]:
"""VF2-SI VERTEX PROBABILITIES"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df = ak.DataFrame({"nodes":ak.array(subgraph_nodes),
                        "color":labels1_subgraph,
                        "value":labels2_subgraph,
                        "True?":labels3_subgraph,
                        "cats":labels4_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)
sg.load_node_attributes(node_df, node_column="nodes", convert_string_labels_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = "and", algorithm_type = "si", 
                                           reorder = True, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with reordering")

We found 0.0 monos inside of the graph with reordering


In [8]:
"""VF2-SI VERTEX AND EDGE PROBABILITIES"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "value":rels1_subgraph 
})
node_df = ak.DataFrame({"nodes":ak.array(subgraph_nodes),
                        "color":labels1_subgraph,
                        "value":labels2_subgraph,
                        "True?":labels3_subgraph,
                        "cats":labels4_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)
sg.load_node_attributes(node_df, node_column="nodes", convert_string_labels_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = "and", algorithm_type = "si", 
                                           reorder = True, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with reordering")

We found 1.0 monos inside of the graph with reordering


In [9]:
"""VF2-PS STRUCTURAL"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = None, algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with VF2-PS")

We found 17.0 monos inside of the graph with VF2-PS


In [10]:
"""VF2-PS VERTEX PROBABILITIES"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph})
node_df = ak.DataFrame({"nodes":ak.array(subgraph_nodes),
                        "color":labels1_subgraph,
                        "value":labels2_subgraph,
                        "True?":labels3_subgraph,
                        "cats":labels4_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)
sg.load_node_attributes(node_df, node_column="nodes", convert_string_labels_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with VF2-PS")

We found 0.0 monos inside of the graph with VF2-PS


In [11]:
"""VF2-PS VERTEX AND EDGE PROBABILITIES"""
edge_df = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph,
                        "value":rels1_subgraph 
})
node_df = ak.DataFrame({"nodes":ak.array(subgraph_nodes),
                        "color":labels1_subgraph,
                        "value":labels2_subgraph,
                        "True?":labels3_subgraph,
                        "cats":labels4_subgraph
})

sg = ar.PropGraph()
sg.load_edge_attributes(edge_df, source_column="src", destination_column="dst", convert_string_relationships_to_categoricals=False)
sg.load_node_attributes(node_df, node_column="nodes", convert_string_labels_to_categoricals=False)

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = "and", algorithm_type = "ps", 
                                           reorder = False, return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with VF2-PS")

We found 1.0 monos inside of the graph with VF2-PS
